In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1.EDA

In [ ]:
train=pd.read_csv("/kaggle/input/sf-crime/train.csv.zip")
test=pd.read_csv("/kaggle/input/sf-crime/test.csv.zip")
train.head(3)

In [ ]:
#데이터타입확인
train.info()
#null값 체크
train.isnull().sum()

In [ ]:
#종속변수 탐색
plt.figure(figsize=(10,5))
train['Category'].value_counts().plot(kind='bar')
# plt.xticks(rotation=-45)

In [ ]:
train[train['Category']=='NON-CRIMINAL'].head()
#범죄를 저지른게 아닌 경우도 존재함. 경찰서 방문 데이터가 다 있는듯 함.
train[train['Category']=='MISSING PERSON'].tail()

In [ ]:
# 범죄 종류 확인
print(set(train['Category']))
print(len(set(train['Category'])))

In [ ]:
#object만 뽑기
category_feature=[col for col in train.columns if train[col].dtypes == 'object']
category_feature.remove('Category')
category_feature.remove('Descript')
category_feature

In [ ]:
import warnings 
warnings.filterwarnings(action='ignore') #경고 제거

#칼럼별로 범죄수 살펴보기 - 요일, 관할경찰, 해결 유무
plt.figure(figsize=(20,10))
for i, col in enumerate(category_feature[1:4]) :
    plt.subplot(1,3,i+1)
    plt.title('{}'.format(col)) #타이틀을 변수명으로 지정
    plt.xticks(rotation=-90) #x축이름 회전
    ft=train[col].value_counts()
    sns.barplot(ft.index,ft.values)

금요일에 건수가 가장 많고, 일요일에 적음. 관할경찰서별로도 차이나는것 확인. 해결 

In [ ]:
set(train['PdDistrict'])

In [ ]:
# 관할구역별 범죄 pie chart 
pd_list=list(set(train['PdDistrict']))
plt.figure(figsize=(15,25))
for i ,col in enumerate(pd_list):
    pd1=train.loc[train['PdDistrict']==col,:]
    plt.subplot(5,2,i+1)
    pd1['Category'].value_counts().plot.pie(autopct='%.1f %%')  #autopct='%.2f %%'
    plt.title('{}'.format(col)) #타이틀을 변수명으로 지정


관할 구역별로 범죄종류의 비율도 다름 확인

In [ ]:
#년도 월 일별 차이 확인
train2=train 
train2['year']=pd.to_datetime(train2["Dates"]).dt.year  #to_datetime : 날짜형식으로 변환
train2['month']=pd.to_datetime(train2["Dates"]).dt.month
train2['day']=pd.to_datetime(train2["Dates"]).dt.day
train2['hour']=pd.to_datetime(train2["Dates"]).dt.hour

In [ ]:
#년도별 범죄수
sns.countplot(train2['year'])

딱히 시간이 지날수록 범죄수가 증가한다거나 감소하는 거 같지는 않다.

In [ ]:
#월별 범죄수
sns.countplot(train2['month'])

In [ ]:
#월별 범죄수
sns.countplot(train2['hour'])

새벽시간에는 범죄율이 감소함. 새벽~오전은 비교적 범죄건수가 낮다

# 2. 변수생성


In [ ]:
train3=train2 
train3_category=[col for col in train3.columns if train3[col].dtypes == 'object']
train3=train3.drop(['Category','Descript'],axis=1)
train3.head(3)

In [ ]:
test.head(3)

In [ ]:
print(len(train3))
print(len(test))

In [ ]:
# 범주형 라벨인코딩
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
pd_total=pd.concat([train3[['DayOfWeek','PdDistrict','Address']],test[['DayOfWeek','PdDistrict','Address']]],axis=0)

In [ ]:
pd_total['le_week']=le.fit_transform(pd_total['DayOfWeek'])
pd_total['le_dstt']=le.fit_transform(pd_total['PdDistrict'])
pd_total['le_addr']=le.fit_transform(pd_total['Address'])

In [ ]:
train4=pd_total.iloc[:874049,:]
test4=pd_total.iloc[874049:,:]

In [ ]:
train4=train4.drop(['DayOfWeek','PdDistrict','Address'],axis=1)

# 3.모델링


In [ ]:
sub=pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')
sub.head()

## KNN
